# Topics – Easy Topic Modeling in Python
# + TATOM (https://liferay.de.dariah.eu/tatom/) 

The text mining technique **Topic Modeling** has become a popular statistical method for clustering documents. This [Jupyter notebook](http://jupyter.org/) introduces a step-by-step workflow, basically containing data preprocessing, the actual topic modeling using **latent Dirichlet allocation** (LDA), which learns the relationships between words, topics, and documents, as well as some interactive visualizations to explore the model.

LDA, introduced in the context of text analysis in [2003](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf), is an instance of a more general class of models called **mixed-membership models**. Involving a number of distributions and parameters, the topic model is typically performed using [Gibbs sampling](https://en.wikipedia.org/wiki/Gibbs_sampling) with conjugate priors and is purely based on word frequencies.

There have been written numerous introductions to topic modeling for humanists (e.g. [this one](http://scottbot.net/topic-modeling-for-humanists-a-guided-tour/)), which provide another level of detail regarding its technical and epistemic properties.

For this workflow, you will need a corpus (a set of texts) as plain text (`.txt`) or [TEI XML](http://www.tei-c.org/index.xml) (`.xml`). Using the `dariah_topics` package, you also have the ability to process the output of [DARIAH-DKPro-Wrapper](https://github.com/DARIAH-DE/DARIAH-DKPro-Wrapper), a command-line tool for *natural language processing*.

Topic modeling works best with very large corpora. The [TextGrid Repository](https://textgridrep.org/) is a great place to start searching for text data. Anyway, to demonstrate the technique, we provide one small text collection in the folder `grenzboten_sample` containing 15 diary excerpts, as well as 15 war diary excerpts, which appeared in *Die Grenzboten*, a German newspaper of the late 19th and early 20th century.

**Of course, you can work with your own corpus in this notebook.**

We're relying on the LDA implementation by [Andrew McCallum](https://people.cs.umass.edu/~mccallum/), called [MALLET](http://mallet.cs.umass.edu/topics.php), which is known to be very robust. Aside from that, we provide two more Jupyter notebooks:

* [IntroducingGensim](IntroducingGensim.ipynb), using LDA by [Gensim](https://radimrehurek.com/project/gensim/), which is attractive because of its multi-core support.
* [IntroducingLda](IntroducingLda.ipynb), using LDA by [lda](http://pythonhosted.org/lda/index.html), which is very lightweight.

For more information in general, have a look at the [documentation](http://dev.digital-humanities.de/ci/job/DARIAH-Topics/doclinks/1/).

## First step: Installing dependencies

To work within this Jupyter notebook, you will have to import the `dariah_topics` library. As you do, `dariah_topics` also imports a couple of external libraries, which have to be installed first. `pip` is the preferred installer program in Python. Starting with Python 3.4, it is included by default with the Python binary installers. If you are interested in `pip`, have a look at [this website](https://docs.python.org/3/installing/index.html).

To install the `dariah_topics` library with all dependencies, open your commandline, go with `cd` to the folder `Topics` and run:

```
pip install -r requirements.txt
```

Alternatively, you can do:

```
python setup.py install
```

If you get any errors or are not able to install *all* dependencies properly, try [Stack Overflow](https://stackoverflow.com/questions/tagged/pip) for troubleshooting or create a new issue on our [GitHub page](https://github.com/DARIAH-DE/Topics).

**Important**: If you are on macOS or Linux, you will have to use `pip3` and `python3`.

### Some final words
As you probably already know, code has to be written in the grey cells. You execute a cell by clicking the **Run**-button (or **Ctrl + Enter**). If you want to run all cells of the notebook at once, click **Cell > Run All** or **Kernel > Restart & Run All** respectively, if you want to restart the Python kernel first. On the left side of an (unexecuted) cell stands `In [ ]:`. The empty bracket means, that the cell hasn't been executed yet. By clicking **Run**, a star appears in the brackets (`In [*]:`), which means the process is running. In most cases, you won't see that star, because your computer is faster than your eyes. You can execute only one cell at once, all following executions will be in the waiting line. If the process of a cell is done, a number appears in the brackets (`In [1]:`).

## Starting with topic modeling!

Execute the following cell to import modules from the `dariah_topics` library.

In [1]:
from cophi_toolbox import preprocessing
from dariah_topics import utils
from dariah_topics import postprocessing
from dariah_topics import visualization

Furthermore, we will need some additional functions from external libraries.

In [2]:
import metadata_toolbox.utils as metadata
import pandas as pd
from pathlib import Path

Let's not pay heed to any warnings right now and execute the following cell.

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import numpy as np

## 1. Preprocessing

### 1.1. Reading a corpus of documents

#### Defining the path to the corpus folder

In the present example code, we are using the 30 diary excerpts from the folder `grenzboten`. To use your own corpus, change the path accordingly.

In [5]:
data = 'Y:/data/projekte/dispecs/topicModelling' 
path_to_corpus = Path(data, 'Die_Zuschauerin/de')

#### Specifying the pattern of filenames for metadata extraction

You have the ability to extract metadata from the filenames. For instance, if your textfiles look like:

```
goethe_1816_stella.txt
```

the pattern would look like this:

```
{author}_{year}_{title}
```

So, let's try this for the example corpus.

In [6]:
pattern = '{name}' #{year}_{author}_{title}_{id}

# Sorting

In [7]:
sorted(os.listdir(path=path_to_corpus))

['Anonym_Die-Zuschauerin_1747.txt']

In [8]:
de_filenames = [os.path.join(path_to_corpus, fn) for fn in sorted(os.listdir(path_to_corpus))]

In [9]:
de_filenames

['Y:\\data\\projekte\\dispecs\\topicModelling\\Die_Zuschauerin\\de\\Anonym_Die-Zuschauerin_1747.txt']

# Chunking in 500 words

In [16]:
def split_text(filename, n_words):
    """Split a text into chunks approximately `n_words` words in length."""
    input = open(filename, 'r', encoding="utf-8")
    words = input.read().split(' ')
    input.close()
    chunks = []
    current_chunk_words = []
    current_chunk_word_count = 0
    for word in words:
        current_chunk_words.append(word)
        current_chunk_word_count += 1
        if current_chunk_word_count == n_words:
            chunks.append(' '.join(current_chunk_words))
            current_chunk_words = []
            current_chunk_word_count = 0
    chunks.append(' '.join(current_chunk_words) )
    return chunks

In [17]:
de_filenames.sort()

In [18]:
chunk_length = 500
chunks = []

In [19]:
for filename in de_filenames:
    chunk_counter = 0
    texts = split_text(filename, chunk_length)
    for text in texts:
        chunk = {'text': text, 'number': chunk_counter, 'filename': filename}
        chunks.append(chunk)
        chunk_counter += 1

In [20]:
len(de_filenames)


1

In [21]:
len(chunks)

104

In [22]:
output_dir = 'Y:/data/projekte/dispecs/topicModelling/Die_Zuschauerin/de_chunks'

for chunk in chunks:
    basename = os.path.basename(chunk['filename'])
    fn = os.path.join(output_dir, "{}{:04d}".format(basename, chunk['number']))
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(chunk['text'])

In [23]:
# umgeändert, sodass valide txt-Dateien als output kommen
for chunk in chunks:
    basename = os.path.basename(chunk['filename'])
    fn_base, fn_ext = os.path.splitext(basename)
    fn = os.path.join(output_dir, "{}_{:04d}{}".format(fn_base, chunk['number'], fn_ext))
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(chunk['text'])

In [24]:
basename = os.path.basename(chunk['filename'])
basename

'Anonym_Die-Zuschauerin_1747.txt'

# Grouping

In [25]:
chunk_path = Path(data, 'Die_Zuschauerin/de_chunks')

In [26]:
de_filenames = [os.path.join(chunk_path, fn) for fn in sorted(os.listdir(chunk_path))]

In [27]:
os.path.basename(filename).split('_')[1]

'Die-Zuschauerin'

In [28]:
authors = [(os.path.basename(filename).split('_')[1]) for filename in de_filenames]

In [ ]:
authors

In [22]:
authors_unique = sorted(set(authors))
authors_unique

['Die-Zuschauerin']

#### Accessing file paths and metadata
We begin by creating a list of all the documents in the folder specified above. That list will tell the function `preprocessing.read_files` (see below) which text documents to read. Furthermore, based on filenames we can create some metadata, e.g. author and title.

In [29]:
meta = pd.concat([metadata.fname2metadata(str(path), pattern=pattern) for path in chunk_path.glob('*.txt')])
meta[:5] # by adding '[:5]' to the variable, only the first 5 elements will be printed

,name
Y:\data\projekte\dispecs\topicModelling\Die_Zuschauerin\de_chunks\Anonym_Die-Zuschauerin_1747_0000.txt,Anonym_Die-Zuschauerin_1747_0000
Y:\data\projekte\dispecs\topicModelling\Die_Zuschauerin\de_chunks\Anonym_Die-Zuschauerin_1747_0001.txt,Anonym_Die-Zuschauerin_1747_0001
Y:\data\projekte\dispecs\topicModelling\Die_Zuschauerin\de_chunks\Anonym_Die-Zuschauerin_1747_0002.txt,Anonym_Die-Zuschauerin_1747_0002
Y:\data\projekte\dispecs\topicModelling\Die_Zuschauerin\de_chunks\Anonym_Die-Zuschauerin_1747_0003.txt,Anonym_Die-Zuschauerin_1747_0003
Y:\data\projekte\dispecs\topicModelling\Die_Zuschauerin\de_chunks\Anonym_Die-Zuschauerin_1747_0004.txt,Anonym_Die-Zuschauerin_1747_0004


In [30]:
type(meta)

pandas.core.frame.DataFrame

In [ ]:
meta[0:1]

#### Read listed documents from folder

In [31]:
corpus = list(preprocessing.read_files(meta.index))
corpus[0][:4000] # printing the first 255 characters of the first document

'\ufeff                                                                                                                                                                                                                                                                                                                                                                                                                                                          Drittes Buch Man handelt meinem Urtheile nach höchst ungerecht, wenn man die Unart und den Fehler derjenigen, die sich zu ungeschickten Gliedern der menschlichen Gesellschaft machen, der Natur zuzuschreiben pflegt. In Engelland geschieht dies sonderlich in einem gewissen Ausdrucke, dessen man sich sehr oft zu bedienen pfleget. He is very ill naturd heißt es gemeiniglich in vielen Umständen'

In [32]:
len(corpus)

104

Your `corpus` contains as much elements (`documents`) as texts in your corpus are. Each element of `corpus` is a list containing exactly one element, the text itself as one single string including all whitespaces and punctuations:

```
[['This is the content of your first document.'],
 ['This is the content of your second document.'],
 ...
 ['This is the content of your last document.']]
```

### 1.3. Tokenize corpus
Now, your `documents` in `corpus` will be tokenized. Tokenization is the task of cutting a stream of characters into linguistic units, simply words or, more precisely, tokens. The tokenize function `dariah_topics` provides is a simple Unicode tokenizer. Depending on the corpus, it might be useful to use an external tokenizer function, or even develop your own, since its efficiency varies with language, epoch and text type.

In [33]:
tokenized_corpus = [list(preprocessing.tokenize(document)) for document in corpus]

At this point, each `document` is represented by a list of separate token strings. As above, have a look at the first document (which has the index `0` as Python starts counting at 0) and show its first 14 words/tokens (that have the indices `0:13` accordingly).

In [34]:
tokenized_corpus[0][0:13]

['drittes',
 'buch',
 'man',
 'handelt',
 'meinem',
 'urtheile',
 'nach',
 'höchst',
 'ungerecht',
 'wenn',
 'man',
 'die',
 'unart']

### 1.4 Create a document-term matrix

The LDA topic model is based on a [document-term matrix](https://en.wikipedia.org/wiki/Document-term_matrix) of the corpus. To improve performance in large corpora, the matrix describes the frequency of terms that occur in the collection. In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms.

#### 1.4.1 Large corpus matrix

If you have a very large corpus, create a document-term matrix designed for large corpora.

In [35]:
document_term_matrix, document_ids, type_ids = preprocessing.create_document_term_matrix(tokenized_corpus,
                                                                                         meta,
                                                                                         large_corpus=True)
(document_term_matrix, document_ids, type_ids)[:5]

(                     0
 document_id type_id   
 1           3176     1
             6063     1
             1318     3
             3960     1
             751      1
             1103     1
             848      1
             6337     1
             2715     1
             5758     1
             5601     2
             3796     1
             4731     1
             4490     1
             1414     1
             4623     1
             4929     2
             4369     2
             6553     1
             2607     1
             1180     2
             5073     1
             3974     1
             6154     1
             4200     1
             6275     1
             970      1
             2258     3
             6078     1
             4689     1
             1780     1
             6429     1
             6326     1
             6120     1
             2624     1
             210      1
             4355     1
             5299     1
             4817     1
             763

#### 1.4.2 Small corpus matrix

Otherwise, use the document-term matrix desigend for small corpora.

In [36]:
document_term_matrix = preprocessing.create_document_term_matrix(tokenized_corpus, meta['name'])
document_term_matrix[:5]

,und,die,sie,zu,in,er,der,so,sich,nicht,...,bloser,canidia,erbrach,erfolg,erlaubet,erwiderten,fächer,gelegentlich,gottesdienstes,ausdrucke
Anonym_Die-Zuschauerin_1747_0000,1.0,2.0,0.0,2.0,3.0,0.0,2.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Anonym_Die-Zuschauerin_1747_0001,14.0,29.0,4.0,12.0,14.0,0.0,12.0,6.0,5.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anonym_Die-Zuschauerin_1747_0002,20.0,13.0,3.0,12.0,4.0,11.0,6.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anonym_Die-Zuschauerin_1747_0003,17.0,11.0,18.0,15.0,9.0,11.0,5.0,9.0,4.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anonym_Die-Zuschauerin_1747_0004,16.0,9.0,9.0,11.0,6.0,10.0,3.0,4.0,3.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


document_term_matrix.columns

document_term_matrix.index

type(document_term_matrix)

dtm_authors = pd.DataFrame(document_term_matrix).groupby(authors).mean().values #SS: statt sum() verwende ich mean()
dtm_authors

type(dtm_authors)

help(pd.DataFrame)

dtm_authors = pd.DataFrame(data=dtm_authors, index=authors_unique, columns=document_term_matrix.columns)

dtm_authors[:10]

### 1.5. Feature removal

*Stopwords* (also known as *most frequent tokens*) and *hapax legomena* are harmful for LDA and have to be removed from the corpus or the document-term matrix respectively. In this example, the 50 most frequent tokens will be categorized as stopwords.

**Hint**: Be careful with removing most frequent tokens, you might remove tokens quite important for LDA. Anyway, to gain better results, it is highly recommended to use an external stopwords list.

In this notebook, we combine the 50 most frequent tokens, hapax legomena and an external stopwordslist.

#### List the 100 most frequent words

If you have chosen the large corpus model, you will have to add `type_ids` to the function `preprocessing.list_mfw()`.

In [37]:
stopwords = preprocessing.list_mfw(document_term_matrix, most_frequent_tokens=100, type_ids=type_ids)

These are the five most frequent words:

In [ ]:
stopwords

#### List hapax legomena

In [38]:
hapax_legomena = preprocessing.find_hapax_legomena(document_term_matrix, type_ids)
print("Total number of types in corpus:", len(type_ids))
print("Total number of hapax legomena:", len(hapax_legomena))

Total number of types in corpus: 6646
Total number of hapax legomena: 5738


#### Optional: Use external stopwordlist

In [39]:
path_to_stopwordlist = Path(data, 'stopwords', 'de.txt')
external_stopwords = [line.strip() for line in path_to_stopwordlist.open('r', encoding='utf-8')]

#### Combine lists and remove content from `tokenized_corpus`

In [40]:
features = stopwords + hapax_legomena + external_stopwords
clean_tokenized_corpus = list(preprocessing.remove_features(features, tokenized_corpus=tokenized_corpus, type_ids=type_ids))

## 2. Model creation

#### Path to MALLET folder 

Now we must tell the library where to find the local instance of MALLET. If you managed to install MALLET, it is sufficient set `path_to_mallet = 'mallet'`, if you store MALLET in a local folder, you have to specify the path to the binary explictly (e.g. `path_to_mallet = 'C:/mallet-2.0.8/bin/mallet'`).

**Whitespaces are not allowed in the path!**

In [41]:
path_to_mallet = 'C:/mallet-2.0.8/bin/mallet.bat'

### 2.1. Create `Mallet` object

Finally, we can instance the `Mallet` object.

In [42]:
Mallet = utils.Mallet(path_to_mallet)

The object `Mallet` has a method `import_tokenized_corpus()` to create a specific corpus file for MALLET.

In [43]:
mallet_corpus = Mallet.import_tokenized_corpus(clean_tokenized_corpus, meta['name']) #meta['year']+'_'+meta['author']+'_'+meta['title']+'_'+meta['id']

In [ ]:
help(Mallet.import_tokenized_corpus)

Furthermore, `Mallet` has the method `train_topics()` to create and train the LDA model. To create a LDA model, there have to be specified a couple of parameters.

But first, if you are curious about any library, module, class or function, try `help()`. This can be very useful, because (at least in a well documented library) explanations of use and parameters will be printed. We're interested in the function `Mallet.train_topics()` in the module `dariah_topics.mallet`, so let's try:

```
help(mallet.Mallet)
```

This will print something like this (in fact even more):

```
Help on method train_topics in module dariah_topics.mallet:

train_topics(mallet_binary, **kwargs) method of dariah_topics.mallet.Mallet instance
    Args:
        input_model (str): Absolute path to the binary topic model created by `output_model`.
        output_model (str): Write a serialized MALLET topic trainer object.
            This type of output is appropriate for pausing and restarting training,
            but does not produce data that can easily be analyzed.
        output_topic_keys (str): Write the top words for each topic and any
            Dirichlet parameters to file.
        topic_word_weights_file (str): Write unnormalized weights for every
            topic and word type.
        word_topic_counts_file (str): Write a sparse representation of topic-word
            assignments. By default this is null, indicating that no file will
            be written.
        output_doc_topics (str): Write the topic proportions per document, at
            the end of the iterations.
        num_topics (int): Number of topics. Defaults to 10.
        num_top_words (int): Number of keywords for each topic. Defaults to 10.
        num_interations (int): Number of iterations. Defaults to 1000.
        num_threads (int): Number of threads for parallel training.  Defaults to 1.
        num_icm_iterations (int): Number of iterations of iterated conditional
            modes (topic maximization).  Defaults to 0.
        no_inference (bool): Load a saved model and create a report. Equivalent
            to `num_iterations = 0`. Defaults to False.
        random_seed (int): Random seed for the Gibbs sampler. Defaults to 0.
        optimize_interval (int): Number of iterations between reestimating
            dirichlet hyperparameters. Defaults to 0.
        optimize_burn_in (int): Number of iterations to run before first
            estimating dirichlet hyperparameters. Defaults to 200.
        use_symmetric_alpha (bool): Only optimize the concentration parameter of
            the prior over document-topic distributions. This may reduce the
            number of very small, poorly estimated topics, but may disperse common
            words over several topics. Defaults to False.
        alpha (float): Sum over topics of smoothing over doc-topic distributions.
            alpha_k = [this value] / [num topics]. Defaults to 5.0.
        beta (float): Smoothing parameter for each topic-word. Defaults to 0.01.
```

So, now you know how to define the number of topics and the number of sampling iterations as well. A higher number of iterations will probably yield a better model, but also increases processing time. `alpha` and `beta` are so-called *hyperparameters*. They influence the model's performance, so feel free to play around with them. In the present example, we will leave the default values. Furthermore, there exist various methods for hyperparameter optimization, e.g. gridsearch or Gaussian optimization.

**Warning: This step can take quite a while!** Meaning something between some seconds and some hours depending on corpus size and the number of iterations. Our example corpus should be done within a minute or two at `num_iterations=1000`.

First, create an output folder:

In [35]:
output = Path(data, 'mallet_output')

if not output.exists():
    output.mkdir()

In [36]:
%%time

Mallet.train_topics(mallet_corpus,
                    output_topic_keys=str(Path(output, 'topic_keys.txt')),
                    output_doc_topics=str(Path(output, 'doc_topics.txt')),
                    num_topics=10,
                    num_iterations=1000,
                    optimize_interval=10)

Wall time: 5.65 s


If you are curious about MALLET's logging, have a look at the file `mallet.log`, which should have been created in the same directory as your notebook is.

### 2.4. Create document-topic matrix

The generated model object can now be translated into a human-readable document-topic matrix (that is a actually a pandas data frame) that constitutes our principle exchange format for topic modeling results. For generating the matrix from a Gensim model, we can use the following function:

In [37]:
topics = postprocessing.show_topics(topic_keys_file=str(Path(output, 'topic_keys.txt')))
topics

,Key 0,Key 1,Key 2,Key 3,Key 4,Key 5,Key 6,Key 7,Key 8,Key 9,Key 10,Key 11,Key 12,Key 13,Key 14,Key 15,Key 16,Key 17,Key 18,Key 19
Topic 0,mutter,belise,rinaldo,kleophil,gerontes,flavia,euphrosyne,rinaldos,flaviens,meinung,antwort,vater,flavien,tausend,verlangen,kinder,worten,hierin,probe,zeitung
Topic 1,art,begebenheiten,natur,himmel,güter,tausend,welt,folgen,leidenschaften,minder,eindruck,tugend,endzweck,bloß,verachtung,eigenschaften,erhalten,sagen,vernunft,verschiedene
Topic 2,lindamire,mercator,eltern,junge,vulpone,palamon,platz,verdacht,sorgfalt,hievon,gethan,konten,reisen,guten,entgegen,bouteillen,loge,vulponen,hoffe,tenderille
Topic 3,vater,neigung,sehen,vorher,verbindung,herzen,glückseligkeit,unglück,schicksal,ursach,hoffnung,familie,zärtlichkeit,vaters,schien,liebte,wolle,angenehme,falls,lieben
Topic 4,sehen,amaranth,treffen,ertragen,zärtlichkeit,amnite,ehre,legen,eindruck,glück,herz,einzig,zeitung,helden,nachricht,schicksal,betrübniß,verwandten,gelegenheit,handlungen
Topic 5,fand,gesellschaft,art,bezeigen,mögte,freunde,hause,geben,wußte,umständen,gelegenheit,sahe,brachte,umstände,ungeachtet,seite,vergnügen,jemals,ungemein,glaubte
Topic 6,herminie,augen,bruder,alkales,hause,polydor,lysimon,merkte,suchte,frauenzimmer,seomanthen,canidien,gesicht,kleidung,eltern,palmyra,schrecken,fenster,eingenommen,arme
Topic 7,personen,leben,gewiß,selbige,welt,unsre,thun,frauenzimmer,geben,stande,jugend,jungen,augen,zeiten,finden,zeigen,gefallen,bringen,natur,ehre
Topic 8,adulphus,mußten,schätze,zimmer,philimont,mögte,empfangen,hand,hoffnung,augen,leben,theils,licht,selbiger,gebäude,traum,erscheinen,versprechen,ruhig,heran
Topic 9,leolin,elmire,lacroon,leolinen,verfahren,panthea,elmirens,fidelio,umstände,welt,pantheen,mutter,finden,suchte,pferde,elmiren,pfund,bevor,personen,güter


## 3. Visualization

Each topic has a certain probability for each document in the corpus (have a look at the cell below). This probability distributions are visualized in an interactive **heatmap** (the darker the color, the higher the probability) which displays the kind of information
                that is presumably most useful to literary scholars. Going beyond pure exploration, this visualization can be used to show thematic developments over a set of texts as well as a single text, akin to a dynamic topic model. What might become
                apparent here, is that some topics correlate highly with a specific author or group of authors, while other topics correlate highly with a specific text or group of texts.

In [38]:
document_topics = postprocessing.show_document_topics(topics=topics,
                                                      doc_topics_file=str(Path(output, 'doc_topics.txt')))
document_topics

,Anonym_Die-Zuschauerin_1747_0000,Anonym_Die-Zuschauerin_1747_0001,Anonym_Die-Zuschauerin_1747_0002,Anonym_Die-Zuschauerin_1747_0003,Anonym_Die-Zuschauerin_1747_0004,Anonym_Die-Zuschauerin_1747_0005,Anonym_Die-Zuschauerin_1747_0006,Anonym_Die-Zuschauerin_1747_0007,Anonym_Die-Zuschauerin_1747_0008,Anonym_Die-Zuschauerin_1747_0009,...,Anonym_Die-Zuschauerin_1747_0094,Anonym_Die-Zuschauerin_1747_0095,Anonym_Die-Zuschauerin_1747_0096,Anonym_Die-Zuschauerin_1747_0097,Anonym_Die-Zuschauerin_1747_0098,Anonym_Die-Zuschauerin_1747_0099,Anonym_Die-Zuschauerin_1747_0100,Anonym_Die-Zuschauerin_1747_0101,Anonym_Die-Zuschauerin_1747_0102,Anonym_Die-Zuschauerin_1747_0103
mutter belise rinaldo,0.0300,0.5916,0.3337,0.0007,0.0007,0.0006,0.3691,0.0775,0.1638,0.0006,...,0.0434,0.0071,0.0006,0.0007,0.0006,0.0061,0.0006,0.0049,0.0300,0.0300
art begebenheiten natur,0.0299,0.0006,0.0006,0.1289,0.3665,0.3372,0.0006,0.0631,0.0159,0.0006,...,0.0007,0.0008,0.0006,0.0007,0.0006,0.0007,0.0371,0.0447,0.0299,0.0299
lindamire mercator eltern,0.2593,0.0003,0.0003,0.0004,0.0004,0.0004,0.0003,0.0099,0.4745,0.0003,...,0.1928,0.4368,0.0003,0.0004,0.0988,0.0004,0.0004,0.0027,0.0167,0.0167
vater neigung sehen,0.3176,0.3330,0.5181,0.6483,0.6242,0.6083,0.3650,0.4964,0.3432,0.3525,...,0.6804,0.5205,0.4558,0.4987,0.4791,0.4986,0.6366,0.5298,0.3176,0.3176
sehen amaranth treffen,0.0097,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.3853,...,0.0002,0.0003,0.0103,0.0002,0.0002,0.0002,0.0002,0.0016,0.0097,0.0097
fand gesellschaft art,0.0172,0.0148,0.0003,0.0004,0.0004,0.0513,0.0004,0.3511,0.0004,0.2587,...,0.0004,0.0004,0.0054,0.0004,0.0003,0.0004,0.0004,0.0028,0.0172,0.0172
herminie augen bruder,0.2642,0.0004,0.1453,0.0005,0.0059,0.0005,0.2326,0.0004,0.0005,0.0004,...,0.0058,0.0006,0.1014,0.0005,0.0004,0.0113,0.0005,0.0035,0.0216,0.0216
personen leben gewiß,0.0210,0.0052,0.0004,0.0005,0.0005,0.0004,0.0307,0.0004,0.0004,0.0004,...,0.0699,0.0132,0.3588,0.4226,0.4041,0.4812,0.3180,0.4015,0.0210,0.2636
adulphus mußten schätze,0.0278,0.0534,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,...,0.0006,0.0007,0.0006,0.0006,0.0006,0.0006,0.0058,0.0046,0.2704,0.2704
leolin elmire lacroon,0.0233,0.0005,0.0005,0.2196,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,...,0.0059,0.0196,0.0661,0.0753,0.0152,0.0005,0.0005,0.0038,0.2659,0.0233


In [ ]:
document_topics.index

In [ ]:
type(document_topics)

# Topics grouped by authors

In [ ]:
document_topics_authors = document_topics.groupby(authors, axis=1).mean().values

In [ ]:
document_topics_authors

In [ ]:
document_topics_authors = pd.DataFrame(data=document_topics_authors, columns=authors_unique, index=document_topics.index)

In [ ]:
document_topics_authors

### 3.1. Distribution of topics

#### Distribution of topics over all documents

The distribution of topics over all documents can now be visualized in an interactive heatmap.

In [39]:
from bokeh.io import output_notebook, show
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [40]:
PlotDocumentTopics = visualization.PlotDocumentTopics(document_topics)
show(PlotDocumentTopics.interactive_heatmap(width=2000, height=600, major_label_text_font_size='10pt'), notebook_handle=True)

Or a static heatmap:

In [ ]:
static_heatmap = PlotDocumentTopics.static_heatmap()
static_heatmap.show()

In [ ]:
static_barchart_per_document = PlotDocumentTopics.static_barchart_per_document(index=2,
                                                                               describer='Topics',
                                                                               )

In [ ]:
help(PlotDocumentTopics)

In [ ]:
static_barchart_per_topic = PlotDocumentTopics.static_barchart_per_topic(index=2, describer='Documents', figsize=(20,15))

In [ ]:
"""
topic_over_time(self, metadata_df, threshold=0.1, starttime=1841, endtime=1920)
 |      Creates a visualization that shows topics over time.
 |      
 |      Description:
 |          With this function you can plot topics over time using metadata stored in the documents name.
 |          Only works with mallet output.
 |      
 |      Args:
 |          metadata_df(pd.Dataframe()): metadata created by metadata_toolbox
 |          threshold(float): threshold set to define if a topic in a document is viable
 |          starttime(int): sets starting point for visualization
 |          endtime(int): sets ending point for visualization
 |      
 |      
 |      Returns: 
 |          matplotlib plot
 |      
 |      Note: this function is created for a corpus with filenames that looks like:
 |              1866_ArticleName.txt
"""
#Funktioniert nicht! 
#topic_over_time = PlotDocumentTopics.topic_over_time(metadata_df=meta, threshold=0.1, starttime=1723, endtime=1765)